In [5]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [15]:
# importieren der Fund Performance Datei wo die alle funds die Preqin finden kann aufgelistet sind. Spalten enthalten u.a. die IRR und wann der Fund eröffnet wurde
df = pd.read_csv(
    "../../data/raw/fund_cashflow.csv",
    low_memory=False  # liest alles auf einmal ein, stabilere Typbestimmung
)
df.shape

(12938, 8)

In [22]:
# importieren der Fund Performance Datei wo die alle funds die Preqin finden kann aufgelistet sind. Spalten enthalten u.a. die IRR und wann der Fund eröffnet wurde
df = pd.read_csv(
    "../../data/raw/cashflows_not_updated.csv",
    low_memory=False  # liest alles auf einmal ein, stabilere Typbestimmung
)
df.shape

(31223, 8)

In [24]:
# transaction_date in Datumsformat umwandeln
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# neuesten Eintrag finden
latest_date = df['transaction_date'].max()

# alle Zeilen mit diesem neuesten Datum anzeigen
df_latest = df[df['transaction_date'] == latest_date]

print(latest_date)
print(df_latest)


2021-12-31 00:00:00
       fund_id transaction_date  firm_id transaction_type  transaction_amount  \
0            3       2021-12-31   152327            Value                   0   
7            9       2021-12-31      741            Value                   0   
15          11       2021-12-31      650            Value                   0   
16          14       2021-12-31        4            Value                   0   
27          15       2021-12-31        4            Value                   0   
...        ...              ...      ...              ...                 ...   
31139   129353       2021-12-31   176053            Value             1235000   
31182   132199       2021-12-31     3795            Value              637440   
31202   133748       2021-12-31    15143            Value            10067400   
31207   135720       2021-12-31   222118            Value            70000000   
31212   136529       2021-12-31    21861            Value             2153770   

       

In [18]:
df.head()

,fund_id,transaction_date,firm_id,transaction_type,transaction_amount,cumulative_contribution,cumulative_distribution,net_cashflow
0,142,2023-12-31,39,Value,69000.00,-9500000.00,8458999.98,-1041000.03
1,195,2023-12-31,6733,Value,0.00,-15526565.52,31162162.62,15635597.10
2,240,2023-12-31,75,Value,0.00,-10629932.98,17172103.76,6542170.78
3,320,2023-12-31,111,Value,26175.29,-12065698.14,28765890.90,16700192.76
4,321,2023-12-31,111,Value,62824.30,-11122641.80,13547354.24,2424712.44


In [19]:
# Anzahl einzigartiger Fund-IDs
unique_funds_count = df["fund_id"].nunique()

print(f"Anzahl einzigartiger Funds: {unique_funds_count}")


Anzahl einzigartiger Funds: 2551


In [21]:
df[df["fund_id"] == 195]

,fund_id,transaction_date,firm_id,transaction_type,transaction_amount,cumulative_contribution,cumulative_distribution,net_cashflow
1,195,2023-12-31,6733,Value,0.0,-15526565.52,31162162.62,15635597.1
10966,195,2024-09-30,6733,Value,0.0,-15526565.52,31162162.62,15635597.1
